In [2]:
import camelot
import pandas as pd
import json
import os
import re
import sys


In [13]:
import camelot
import pandas as pd
import json
import os
import re
import sys

def extract_tables_from_pdf(pdf_path, output_json_path):
    """
    Extrae tablas de un archivo PDF y las guarda en un archivo JSON.
    Este script está optimizado para la estructura del PDF de GENTECH y
    ahora incluye una lógica para detectar y categorizar los productos,
    manejando celdas combinadas de forma más robusta.

    Args:
        pdf_path (str): La ruta del archivo PDF de entrada.
        output_json_path (str): La ruta donde se guardará el archivo JSON de salida.
    """
    # Verificar si el archivo PDF existe
    if not os.path.exists(pdf_path):
        print(f"Error: El archivo PDF '{pdf_path}' no se encontró.")
        return

    try:
        # 1. Extracción con coordenadas específicas
        print(f"🕵️‍♂️ Extrayendo tablas de '{pdf_path}' usando coordenadas y el método 'stream'...")
        tables = camelot.read_pdf(pdf_path, flavor='stream', pages='1', table_areas=['50,780,800,0'])

        if tables.n == 0:
            print("❌ No se encontraron tablas. Verifique las coordenadas o el tipo de tabla.")
            return

        all_tables_data = []

        # 2. Transformación y Limpieza de la tabla extraída
        for i, table in enumerate(tables):
            df = table.df.copy()
            
            # Limpiar los datos de la tabla, eliminando espacios extra
            df = df.map(lambda x: str(x).strip() if isinstance(x, str) else x)

            # Eliminar filas completamente vacías
            df.dropna(how='all', inplace=True)
            
            # Eliminar la fila de encabezado que a veces se extrae
            df = df[~df.iloc[:, 0].str.contains('DESCRIPCIÓN', na=False)]
            
            # Reorganizar los encabezados y datos
            df.columns = ['DESCRIPCIÓN', 'PRESENTACIÓN', 'CONTENIDO NETO', 'CANTIDAD POR BULTO', 'PRECIO UNITARIO NETO', 'PRECIO UNITARIO CON IVA']

            products = []
            current_category = 'Sin Categoria'
            last_product_desc = ""

            for idx, row in df.iterrows():
                row_values_str = ' '.join(str(val) for val in row.tolist()).upper()
                
                # Detectar las filas de categorías
                category_keywords = ['LINEA ALTO RENDIMIENTO', 'LINEA PREMIUM', 'LINEA IRON', 'LINEA BEAUTY', 'LINEA PRE_WORKOUT', 'LINEA NUTRICION',
                                      'LINEA KIDS', 'LINEA VEGGIE PLANT BASED', 'ACCESORIOS']
                is_category_row = any(keyword in row_values_str for keyword in category_keywords)

                if is_category_row:
                    category_name = next((val for val in row.tolist() if str(val).strip()), None)
                    if category_name:
                        cleaned_name = re.sub(r'(LÍNEA|LINEA)\s*', '', str(category_name), flags=re.IGNORECASE).strip()
                        if cleaned_name.upper() not in ['DESCRIPCIÓN', 'PRESENTACIÓN', 'CONTENIDO NETO', 'CANTIDAD POR BULTO', 'PRECIO UNITARIO NETO', 'PRECIO UNITARIO CON IVA']:
                            current_category = cleaned_name
                    continue

                # Si la fila tiene una descripción no vacía, es un producto principal
                if str(row['DESCRIPCIÓN']).strip():
                    last_product_desc = str(row['DESCRIPCIÓN'])

                # Comprobar si la fila contiene múltiples entradas de producto debido a celdas combinadas
                # y dividir todas las columnas relevantes por el salto de línea
                if '\n' in str(row['PRESENTACIÓN']) or '\n' in str(row['CONTENIDO NETO']) or '\n' in str(row['PRECIO UNITARIO CON IVA']):
                    presentaciones = str(row['PRESENTACIÓN']).split('\n')
                    contenidos = str(row['CONTENIDO NETO']).split('\n')
                    cantidades = str(row['CANTIDAD POR BULTO']).split('\n')
                    precios_netos = str(row['PRECIO UNITARIO NETO']).split('\n')
                    precios_iva = str(row['PRECIO UNITARIO CON IVA']).split('\n')

                    # Iterar sobre las partes divididas para crear un producto por cada una
                    # Ahora se usa el largo de la lista de precios_iva, que es el más fiable
                    for j in range(len(precios_iva)):
                        product_row = {
                            'Categoria': current_category,
                            'DESCRIPCIÓN': last_product_desc,  # Asignamos la última descripción válida
                            'PRESENTACIÓN': presentaciones[j].strip() if j < len(presentaciones) else '',
                            'CONTENIDO NETO': contenidos[j].strip() if j < len(contenidos) else '',
                            'CANTIDAD POR BULTO': cantidades[j].strip() if j < len(cantidades) else '',
                            'PRECIO UNITARIO NETO': precios_netos[j].strip() if j < len(precios_netos) else '',
                            'PRECIO UNITARIO CON IVA': precios_iva[j].strip()
                        }
                        products.append(product_row)
                
                # Si es una fila de producto normal (sin celdas combinadas), lo procesamos
                elif str(row['PRECIO UNITARIO CON IVA']).strip():
                    product_row = {
                        'Categoria': current_category,
                        'DESCRIPCIÓN': last_product_desc,
                        'PRESENTACIÓN': row['PRESENTACIÓN'],
                        'CONTENIDO NETO': row['CONTENIDO NETO'],
                        'CANTIDAD POR BULTO': row['CANTIDAD POR BULTO'],
                        'PRECIO UNITARIO NETO': row['PRECIO UNITARIO NETO'],
                        'PRECIO UNITARIO CON IVA': row['PRECIO UNITARIO CON IVA']
                    }
                    products.append(product_row)
            
            all_tables_data.append({"tabla_1": products})

            return products

        # # 3. Carga: Escribir los datos en el archivo JSON
        # with open(output_json_path, 'w', encoding='utf-8') as f:
        #     json.dump(all_tables_data, f, ensure_ascii=False, indent=4)
        
        # print(f"✅ ¡Éxito! {tables.n} tablas extraídas y guardadas en '{output_json_path}'.")

    except Exception as e:
        print(f"❌ Ocurrió un error inesperado: {e}")
        # En caso de error, imprimir el DataFrame para depuración
        try:
            print("--- Contenido del DataFrame para depuración ---")
            print(df)
            print("---------------------------------------------")
        except NameError:
            pass


In [14]:
pdf_file_path = 'LISTA DE PRECIOS MAYORISTA - GENTECH - SEPTIEMBRE 2025.pdf'
output_json_file = 'test.json'

tablas = extract_tables_from_pdf(pdf_file_path, output_json_file)
tablas

🕵️‍♂️ Extrayendo tablas de 'LISTA DE PRECIOS MAYORISTA - GENTECH - SEPTIEMBRE 2025.pdf' usando coordenadas y el método 'stream'...


[{'Categoria': 'Sin Categoria',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': '',
  'CONTENIDO NETO': '',
  'CANTIDAD POR BULTO': '',
  'PRECIO UNITARIO NETO': '',
  'PRECIO UNITARIO CON IVA': 'IVA'},
 {'Categoria': 'ALTO RENDIMIENTO',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': 'POTE -150 COMPRIMIDOS',
  'CONTENIDO NETO': '285',
  'CANTIDAD POR BULTO': '12',
  'PRECIO UNITARIO NETO': '$',
  'PRECIO UNITARIO CON IVA': '$'},
 {'Categoria': 'ALTO RENDIMIENTO',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': '',
  'CONTENIDO NETO': '',
  'CANTIDAD POR BULTO': '',
  'PRECIO UNITARIO NETO': '11.424,79',
  'PRECIO UNITARIO CON IVA': '13.824,00'},
 {'Categoria': 'ALTO RENDIMIENTO',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': 'POTE - 325 COMPRIMIDOS',
  'CONTENIDO NETO': '618',
  'CANTIDAD POR BULTO': '12',
  'PRECIO UNITARIO NETO': '$',
  'PRECIO UNITARIO CON IVA': '$'},
 {'Categoria': 'ALTO RENDIMIENTO',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': '',
  'CONTENIDO NETO': '',
  'CANTIDAD POR BULTO': '',
  'PRECIO UNITARIO 

In [ ]:
pdf_path = 'LISTA DE PRECIOS MAYORISTA - GENTECH - SEPTIEMBRE 2025.pdf'
tables = camelot.read_pdf(pdf_path, flavor='stream', pages='1', table_areas=['50,780,800,0'])


<TableList n=1>